<a href="https://colab.research.google.com/github/JhonFiUNFV/python_prep/blob/master/05_backpropagation_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clonamos el repositorio para obtener los dataSet

In [ ]:
!git clone https://github.com/joanby/tensorflow.git

Cloning into 'tensorflow'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 60311 (delta 32), reused 37 (delta 23), pack-reused 60260
Receiving objects: 100% (60311/60311), 442.46 MiB | 36.15 MiB/s, done.
Resolving deltas: 100% (82/82), done.
Checking out files: 100% (60225/60225), done.


# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [ ]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

##Especificando la versión de TensorFlow

Ejecutando "importar tensorflow" importará la versión por defecto (actualmente 2.x). Puedes usar la 1.x ejecutando una celda con la "versión mágica de tensorflow" **antes de ejecutar "importar tensorflow".

### Si no funciona hacer el pip install


In [ ]:
#!pip install tensorflow==1.14
%tensorflow_version 1.x

TensorFlow 1.x selected.


# Importar Tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt

1.15.2


In [ ]:
session = tf.Session()

# Propagación hacia atrás

### Ejemplo de Regresión
- $X \sim N(1.0, 0.1)$
- $y = Ax, A = 10$
- target = 10
- $L2$ función de pérdidas

In [ ]:
import numpy as np
x_vals = np.random.normal(loc=1, scale=0.1, size=200)
y_vals = np.repeat(10., 200)

In [ ]:
x_data = tf.placeholder(shape = [1], dtype=tf.float32)
y_target = tf.placeholder(shape = [1], dtype=tf.float32)
A = tf.Variable(tf.random_normal(shape=[1]))

In [ ]:
my_pred = tf.multiply(A, x_data)

In [ ]:
loss = tf.square(my_pred - y_target)

In [ ]:
my_optim = tf.train.GradientDescentOptimizer(learning_rate=0.025)
train_step = my_optim.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
session.run(init)

In [ ]:
for i in range(200):
    rand_index = np.random.choice(200)
    rand_x = [x_vals[rand_index]]
    rand_y = [y_vals[rand_index]]
    session.run(train_step, feed_dict={x_data : rand_x, y_target : rand_y})
    if (i+1)%20==0:
        print("Paso #"+str(i+1)+" A = "+str(session.run(A)) + 
              " Loss: "+str(session.run(loss, feed_dict={x_data:rand_x, y_target: rand_y})))

Paso #20 A = [6.2592483] Loss: [6.3893924]
Paso #40 A = [8.640756] Loss: [1.5384533]
Paso #60 A = [9.115492] Loss: [2.637954]
Paso #80 A = [9.528063] Loss: [1.5413789]
Paso #100 A = [9.979744] Loss: [1.167261]
Paso #120 A = [9.956358] Loss: [0.7251246]
Paso #140 A = [9.992054] Loss: [0.15124306]
Paso #160 A = [9.962452] Loss: [0.540453]
Paso #180 A = [9.793145] Loss: [0.02940218]
Paso #200 A = [9.694743] Loss: [0.5770091]


### Ejemplo de clasificación binaria
- $X_1\sim N(-2, 1), X_2 \sim N(2,1)$
- $target(X_1) = 0, target(X_2) = 1$
- sigmoid(x+A) = $\frac{1}{1+e^{-(x+A)}}$
- Determinar el valor de $A$
- Teóricamente $A\simeq \frac{m_1+m_2}{2}, m_1 = -2, m_2 = 2$

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
session = tf.Session()

In [ ]:
x_vals = np.concatenate((np.random.normal(-2,1,100), np.random.normal(2,1,100)))
y_vals = np.concatenate((np.repeat(0., 100), np.repeat(1., 100)))

In [ ]:
x_data = tf.placeholder(shape = [1], dtype = tf.float32)
y_target = tf.placeholder(shape = [1], dtype = tf.float32)
A = tf.Variable(tf.random_normal(mean = 10, shape=[1]))

In [ ]:
my_prediction = tf.add(x_data, A)

In [ ]:
my_prediction_expanded = tf.expand_dims(my_prediction, 0)
y_target_expanded = tf.expand_dims(y_target,0 )

In [ ]:
init = tf.initialize_all_variables()
session.run(init)
print(session.run(A))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[8.743118]


In [ ]:
xentr = tf.nn.sigmoid_cross_entropy_with_logits(logits=my_prediction_expanded, labels=y_target_expanded)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
my_optim = tf.train.GradientDescentOptimizer(learning_rate=0.04)
train_step = my_optim.minimize(xentr)

In [ ]:
for i in range(2000):
    rand_idx = np.random.choice(200)
    rand_x = [x_vals[rand_idx]]
    rand_y = [y_vals[rand_idx]]
    session.run(train_step, feed_dict={x_data : rand_x, y_target : rand_y})
    if (i+1)%100==0:
        print("Paso #"+str(i+1)+", A = "+str(session.run(A))+", Loss = "+
             str(session.run(xentr, feed_dict={x_data : rand_x, y_target: rand_y})))

Paso #100, A = [6.913806], Loss = [[5.2362423]]
Paso #200, A = [5.21407], Loss = [[0.00050641]]
Paso #300, A = [3.6885629], Loss = [[0.00386316]]
Paso #400, A = [2.4036148], Loss = [[0.9942923]]
Paso #500, A = [1.4065174], Loss = [[0.36287558]]
Paso #600, A = [0.97767586], Loss = [[0.15852872]]
Paso #700, A = [0.714153], Loss = [[0.47719264]]
Paso #800, A = [0.55385613], Loss = [[0.11512167]]
Paso #900, A = [0.44637236], Loss = [[0.24630712]]
Paso #1000, A = [0.39098334], Loss = [[0.14625303]]
Paso #1100, A = [0.24433221], Loss = [[0.10257504]]
Paso #1200, A = [0.20848513], Loss = [[0.05749148]]
Paso #1300, A = [0.2223705], Loss = [[0.17041756]]
Paso #1400, A = [0.22274321], Loss = [[0.3280766]]
Paso #1500, A = [0.23481074], Loss = [[0.08052165]]
Paso #1600, A = [0.29480335], Loss = [[0.01330733]]
Paso #1700, A = [0.19204144], Loss = [[0.12708126]]
Paso #1800, A = [0.20460548], Loss = [[0.16925547]]
Paso #1900, A = [0.22687757], Loss = [[0.4639144]]
Paso #2000, A = [0.03422413], Loss =